In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xgi

## Notebook 1

In [ ]:
H = xgi.Hypergraph()
H.add_nodes_from(["A", "B", "C", "D"])
H.add_edges_from([{"A", "B"}, {"A", "B", "C"}])
print(H)

In [ ]:
DH = xgi.DiHypergraph()
DH.add_edge([{"A", "B"}, {"C", "D"}])
print(DH)

## Notebook 2

In [ ]:
H = xgi.load_xgi_data("contact-high-school")
m1 = H.num_edges

H.cleanup(
    isolates=True,
    singletons=True,
    multiedges=False,
    connected=False,
    relabel=False,
    in_place=True,
)
m2 = H.num_edges

multiplicity = m1 / m2
print(multiplicity)

In [ ]:
H = xgi.load_xgi_data("congress-bills", max_order=10)
xgi.write_edgelist(H, "edgelist.txt", delimiter="\t")

## Notebook 3

In [ ]:
H = xgi.load_xgi_data("kaggle-whats-cooking")

In [ ]:
len(H.edges.filterby("size", 5, "gt"))

In [ ]:
len(H.nodes.filterby("degree", 1))

In [ ]:
sizes = H.edges.size.asnumpy()
plt.hist(sizes, bins=np.arange(60))
plt.show()

In [ ]:
@xgi.nodestat_func
def user_degree(net, bunch):
    """The user degree of a bunch of nodes in net."""
    return {
        n: sum(1.0 / net.edges.size[i] for i in net.nodes.memberships(n)) for n in bunch
    }

In [ ]:
H.nodes.user_degree.asdict()

In [ ]:
H.nodes.degree.asdict()

In [ ]:
print(
    f"The minimum and maximum number of ingredients are {H.edges.size.min()} and {H.edges.size.max()}, respectively"
)

In [ ]:
d = H.nodes.degree.asdict()
id = max(d, key=d.get)
print(f"The most popular ingredient is {H.nodes[id]['name']}.")

## Notebook 4

In [ ]:
H = xgi.load_xgi_data("diseasome")
edge_sizes = xgi.unique_edge_sizes(H)

data = dict()
for s in edge_sizes:
    ids = H.edges.filterby("size", s)
    data[s] = xgi.subhypergraph(H, edges=ids)

In [ ]:
for s, h in data.items():
    print(s)
    print(xgi.unique_edge_sizes(h))

In [ ]:
H = xgi.load_xgi_data("email-eu")
H.cleanup()
D = H.dual()

In [ ]:
print(f"The number of singletons is {len(D.edges.singletons())}")
print(f"The number of isolates is {len(D.nodes.isolates())}")
print(f"The number of multiedges is {len(D.edges.duplicates())}")
print(f"Is the dual connected? {xgi.is_connected(D)}")

In [ ]:
H = xgi.load_xgi_data("hospital-lyon")
H.cleanup()
SC = xgi.SimplicialComplex(H)

L = xgi.hodge_laplacian(SC, order=2)

In [ ]:
plt.spy(L)
plt.xlabel("order-2 simplices")
plt.ylabel("order-2 simplices")

In [ ]:
H = xgi.load_xgi_data("kaggle-whats-cooking")
cuisines = set(H.edges.attrs("name").aslist())
subhypergraphs = dict()

print(cuisines)

for c in cuisines:
    subhypergraphs[c] = xgi.subhypergraph(H, edges=H.edges.filterby_attr("name", c))

In [ ]:
num_unused_ingredients = dict()
most_used_ingredient = dict()
num_components = dict()

for c in cuisines:
    num_unused_ingredients[c] = len(subhypergraphs[c].nodes.isolates())
    d = subhypergraphs[c].nodes.degree.asdict()
    id = max(d, key=d.get)
    most_used_ingredient[c] = H.nodes[id]["name"]
    num_components[c] = xgi.number_connected_components(subhypergraphs[c])

In [ ]:
print("Number of unused ingredients:")
print(num_unused_ingredients)
print("Most used ingredient:")
print(most_used_ingredient)
print("Number of connected components:")
print(num_components)

## Notebook 5

In [ ]:
H = xgi.uniform_HPPM(100, 3, 8, 0.95)

pos = xgi.pca_transform(xgi.pairwise_spring_layout(H))
xgi.draw(H, pos=pos, node_fc=H.nodes.clustering_coefficient, node_shape="s")

## Notebook 6

In [ ]:
n = 100

connected = False
while not connected:
    H = xgi.fast_random_hypergraph(n, [0.05, 0.001], seed=None)
    connected = xgi.is_connected(H)
omega = 2 * np.random.normal(1, 0.05, n)
theta0 = np.linspace(0, 2 * np.pi, n)
timesteps = 500
dt = 0.01

k2list = np.linspace(0, 3, 10)
R = []

for k2 in k2list:
    theta, _ = xgi.simulate_kuramoto(
        H, k2=k2, k3=2, omega=omega, theta=theta0, timesteps=timesteps, dt=dt
    )
    R.append(xgi.compute_kuramoto_order_parameter(theta)[-1])

In [ ]:
plt.plot(k2list, R)
plt.xlabel(r"$k_2$")
plt.ylabel(r"$R$")